In [4]:

import cv2
import numpy as np
import os
from scipy import ndimage
from collections import Counter
from scipy.stats import skew, kurtosis

def calculate_features(img_vector):
    """Calculate statistical features from the image vector."""
    if np.all(img_vector == img_vector[0]):
        return np.array([0.0, 0.0, 0.0, 0.0])  # Return zeros for uniform images

    skewness = skew(img_vector)
    kurt = kurtosis(img_vector)

    crest_factor = np.max(img_vector) / (np.mean(img_vector) + 1e-9)
    rms_value = np.sqrt(np.mean(np.square(img_vector)))
    shape_factor = rms_value / (np.mean(img_vector) + 1e-9)
    impulse_factor = np.max(img_vector) / rms_value

    return np.array([skewness, kurt, crest_factor, shape_factor, impulse_factor])

def process_image(img_path):
    """Process the image to extract features."""
    try:
        img = cv2.imread(img_path)
        if img is None:
            raise FileNotFoundError(f"Image could not be read: {img_path}")

        gray_img = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), (128, 128))
        img_derivative = np.abs(ndimage.sobel(gray_img, axis=0, mode='constant')) + \
                         np.abs(ndimage.sobel(gray_img, axis=1, mode='constant'))

        img_vector = (img_derivative - np.min(img_derivative)) / (np.max(img_derivative) - np.min(img_derivative))
        img_vector = img_vector.flatten().astype(np.float32)

        features = calculate_features(img_vector)
        return img_vector, features

    except Exception as processing_error:
        print(f"Error processing image: {processing_error}")
        return None, None

def load_data(data_folders):
    """Load images and their labels from the specified directories."""
    x_data, y_data, feature_data = [], [], []

    for label, folder_path in enumerate(data_folders, start=1):
        if not os.path.exists(folder_path):
            print(f"Folder does not exist: {folder_path}")
            continue

        print(f"Loading data from: {folder_path}")
        for filename in os.listdir(folder_path):
            image_path = os.path.join(folder_path, filename)
            vector, features = process_image(image_path)
            if vector is not None:
                x_data.append(vector)
                y_data.append(label)
                feature_data.append(features)

    return np.array(x_data), np.array(y_data), np.array(feature_data)

def calculate_distance(a, b):
    """Calculate Euclidean distance between two vectors."""
    return np.sqrt(np.sum((a - b) ** 2))

def KNN(x_train, y_train, sample_test, k=5):
    """K-Nearest Neighbors algorithm for classification with weighted voting."""
    if k > len(x_train):
        print("K cannot be greater than the number of training samples.")
        k = len(x_train)

    distances = [calculate_distance(sample_test, x) for x in x_train]
    nearest_indices = np.argsort(distances)[:k]

    nearest_labels = [y_train[i] for i in nearest_indices]
    nearest_distances = [distances[i] for i in nearest_indices]

    # Create a dictionary to count votes
    vote_count = {}
    for label, distance in zip(nearest_labels, nearest_distances):
        weight = 1 / (distance + 1e-9)  # Inverse of distance as weight
        if label not in vote_count:
            vote_count[label] = 0
        vote_count[label] += weight

    # Get the label with the highest weighted vote
    predicted_label = max(vote_count, key=vote_count.get)
    return predicted_label

# Paths for training data folders
folders = [
    "/content/train/camera",
    "/content/train/flamingo",
    "/content/train/pizza"
]

try:
    x_train, y_train, feature_data_train = load_data(folders)
    print(f"x_train shape: {x_train.shape}, y_train shape: {y_train.shape}, feature_data_train shape: {feature_data_train.shape}")

    if x_train.shape[0] == 0 or y_train.shape[0] == 0:
        raise Exception("No training data loaded!")

    # Test phase
    test_folders = {
        "camera": "/content/test/camera",
        "flamingo": "/content/test/flamingo",
        "pizza": "/content/test/pizza"
    }

    total_images, correct_predictions = 0, 0

    for category, test_folder_path in test_folders.items():
        print(f"Testing on: {category} folder")
        try:
            for filename in os.listdir(test_folder_path):
                test_image_path = os.path.join(test_folder_path, filename)
                test_sample_vector, test_features = process_image(test_image_path)

                if test_sample_vector is not None:
                    predicted_label = KNN(x_train, y_train, test_sample_vector, k=5)
                    expected_label = 1 if category == "camera" else 2 if category == "flamingo" else 3

                    print(f"{filename} - Expected: {expected_label}, Predicted: {predicted_label}")

                    total_images += 1
                    if predicted_label == expected_label:
                        correct_predictions += 1

        except Exception as test_error:
            print(f"Error in test images: {test_error}")

    if total_images > 0:
        accuracy = (correct_predictions / total_images) * 100
        print(f"Accuracy: %{accuracy:.2f}")
    else:
        print("No images were tested.")

except Exception as data_loading_error:
    print(f"Error loading data: {data_loading_error}")

Loading data from: /content/train/camera
Loading data from: /content/train/flamingo
Loading data from: /content/train/pizza
x_train shape: (135, 16384), y_train shape: (135,), feature_data_train shape: (135, 5)
Testing on: camera folder
image_0037.jpg - Expected: 1, Predicted: 1
image_0046.jpg - Expected: 1, Predicted: 2
image_0040.jpg - Expected: 1, Predicted: 1
image_0035.jpg - Expected: 1, Predicted: 3
image_0024.jpg - Expected: 1, Predicted: 1
Testing on: flamingo folder
image_0022.jpg - Expected: 2, Predicted: 2
image_0041.jpg - Expected: 2, Predicted: 3
image_0059.jpg - Expected: 2, Predicted: 2
image_0050.jpg - Expected: 2, Predicted: 3
image_0051.jpg - Expected: 2, Predicted: 2
image_0024.jpg - Expected: 2, Predicted: 2
image_0047.jpg - Expected: 2, Predicted: 2
image_0043.jpg - Expected: 2, Predicted: 2
Testing on: pizza folder
image_0037.jpg - Expected: 3, Predicted: 2
image_0038.jpg - Expected: 3, Predicted: 2
image_0046.jpg - Expected: 3, Predicted: 2
image_0040.jpg - Expec

In [3]:
# prompt: unzip files

!unzip test.zip
!unzip train.zip

Archive:  test.zip
   creating: test/
   creating: test/camera/
  inflating: test/camera/image_0024.jpg  
  inflating: test/camera/image_0035.jpg  
  inflating: test/camera/image_0037.jpg  
  inflating: test/camera/image_0040.jpg  
  inflating: test/camera/image_0046.jpg  
   creating: test/flamingo/
  inflating: test/flamingo/image_0022.jpg  
  inflating: test/flamingo/image_0024.jpg  
  inflating: test/flamingo/image_0041.jpg  
  inflating: test/flamingo/image_0043.jpg  
  inflating: test/flamingo/image_0047.jpg  
  inflating: test/flamingo/image_0050.jpg  
  inflating: test/flamingo/image_0051.jpg  
  inflating: test/flamingo/image_0059.jpg  
   creating: test/pizza/
  inflating: test/pizza/image_0024.jpg  
  inflating: test/pizza/image_0035.jpg  
  inflating: test/pizza/image_0037.jpg  
  inflating: test/pizza/image_0038.jpg  
  inflating: test/pizza/image_0040.jpg  
  inflating: test/pizza/image_0046.jpg  
Archive:  train.zip
   creating: train/
   creating: train/camera/
  inflat